In [ ]:
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/"+SANDBOX_NAME+"/data/"

In [ ]:
!apt-get install openjdk-8-jdk -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init() # SPARK_HOME

from pyspark.ml.feature import VectorAssembler



# Spark ML Transformación de Variables

Cargamos un dataset con información sobre cuán seguro es un coche. Con este dataset se estudiarán funciones muy importantes de Spark ML.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




### Crear SparkSession
Nota: en DATIO no es necesario crear la SparkSession ya que al iniciar un notebook con el Kernel PySpark Python3 - Spark 2.1.0  se crea automáticamente.

In [ ]:
# Respuesta aqui
DATA_PATH = 'drive/MyDrive/2021Q1_DSF_contents/DATA/'
cars = spark.read.csv(DATA_PATH+'automobile.csv', sep=';', header=True, inferSchema=True)

cars.printSchema()

root
 |-- normalized_losses: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- aspiration: string (nullable = true)
 |-- num_of_doors: string (nullable = true)
 |-- body_style: string (nullable = true)
 |-- drive_wheels: string (nullable = true)
 |-- engine_location: string (nullable = true)
 |-- wheel_base: double (nullable = true)
 |-- length: double (nullable = true)
 |-- width: double (nullable = true)
 |-- height: double (nullable = true)
 |-- curb_weight: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- num_of_cylinders: string (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- fuel_system: string (nullable = true)
 |-- bore: double (nullable = true)
 |-- stroke: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- peak_rpm: integer (nullable = true)
 |-- city_mpg: integer (nullable = true)
 |-- highway_mpg: intege



### Cargar datos y comprobar schema

El método _read.csv_ tiene un parámetro _inferSchema_. El mismo permite inferir el tipo de las columnas, para ello requiere recorrer una vez más los datos y por defecto es _False_.

In [ ]:
# Respuesta aqui




### VectorAssembler



Un _VectorAssembler_ es un transformador de múltiples características ( _features_ ) en una sola columna de tipo vector. Lo construiremos con todas las variables menos con la columna objetivo 'symboling'.

In [ ]:
# Respuesta aqui



Estudiando el error se lee:
    **IllegalArgumentException: 'Data type StringType is not supported.'**
    
Recordamos que VectorAssembler solo acepta los siguientes tipos de datos:

- numéricos
- booleanos
- vector
    



Estudiamos el tipo de cada una de las variables y hacemos VectorAssembler para todas las variables cuyos tipos sí están permitidos. Es decir el _VectorAssembler_ no debe incluir columnas de tipo _string_.

In [ ]:
# Respuesta aqui


In [ ]:
# Respuesta aqui




Ha vuelto a fallar, ¿qué ocurre?

En la version de Spark 2.1 el mensaje no parece aportar muchos indicios  que el error. Sin embargo, en la version de Spark 2.2  el error se describe de la siguiente manera:
    
**Caused by: org.apache.spark.SparkException: Values to assemble cannot be null.**

Así pues, se tiene que se deben haber filtrado correctamente los valores nulos antes de crear un VectorAssembler.




Quitaremos todas las filas con nulos:

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui



**¡¡Ahora se ha podido crear el VectorAssembler!!**

Pero únicamente se han seleccionado aquellas variables que no son de tipo string. 



### StringIndexer



* Hagamos StringIndexer para la variable 'make' que representa la marca del auto, cogiendo el dataset inicial, `cars`

In [ ]:
# Respuesta aqui



De nuevo se produce un error. En la versión de Spark 2.1 el mensaje no parece aportar muchos indicios acerca del mismo.
En la versión de Spark 2.2 el error dice lo siguiente: **Caused by: org.apache.spark.SparkException: StringIndexer encountered NULL value. To handle or skip NULLS, try setting StringIndexer.handleInvalid.**

Es importante haber tratado correctamente los nulos antes.

¿Qué desventaja tendría utilizar handleInvalid tal como se indica?

In [ ]:
# Respuesta aqui



Si se accede a `feature_indexer_model.labels` se obtiene un vector construido por `StringIndexer`. El vector está ordenado por la frecuencia de los valores, por lo tanto el valor más frecuente tiene índice 0.

In [ ]:
# Respuesta aqui



¿Qué más variables se pueden transformar con StringIndexer para ser incluidas en los modelos de Machine Learning? Ojo con `num_of_doors`. Transforma todas las demás restantes y actualiza el dataset sin nulos.

In [ ]:
cars_indexed.dtypes

In [ ]:
categorical_columns = [x[0] for x in cars_indexed.dtypes if x[1] in ['string', 'bool'] and x[0]!='make']
categorical_columns

In [ ]:
for x in categorical_columns:
    print(x)
    feature_indexer = StringIndexer(inputCol=x, outputCol=x+'_indexed')

    feature_index = feature_indexer.fit(cars_indexed) # Please bear in mind, now we are using cars_no_nulls
    cars_indexed = feature_index.transform(cars_indexed)



In [ ]:
cars_indexed.show(5)



### CountVectorizer



* Hagamos CountVectorizer para la variable 'num_of_doors'. 

| num_of_doors   |
| -------------: |
| [four]| 
| [two,four]     | 


In [ ]:
# Respuesta aqui



Mirando el schema se ve que 'num_of_doors' no tiene el formato correcto (es de tipo _string_). Vamos a convertirlo a _ArrayType(StringType())_

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui



Volvamos a probar otra vez:

In [ ]:
# Respuesta aqui



En la siguiente tabla se puede apreciar la conversión realizada con _CountVectorizer_

| num_of_doors   | doors_counter   |
| -------------: | -------------: |
| [four]| (2,[0],[1.0]) |
| [two,four]     | (2,[0,1],[1.0,1.0])|

La columna *doors_counter* contiene un _CountVectorizerModel_ que es un vector con tres campos. El primero indica la cantidad de valores posibles que tiene la columna *num_of_doors*, en este caso es 2. El segundo campo indica los índices del vector donde se han encontrado entradas con un valor diferente de cero. El tercer campo indica qué números se encuentran en esos índices. Se puede saber con *model_cv.vocabulary* que 'four' corresponde a que en el índice 0 del vector haya un 1.0 (el vector de 2 posiciones sería [0, 1]), y 'two' corresponde a que en la posición 1 del vector haya un 1.0 (el vector de dos posiciones sería [1, 0])





### OneHotEncoder



* Hagamos OneHotEncoder para la variable 'make' (recordar que contiene las marcas de distintos autos)

In [ ]:
# Respuesta aqui



Salta el siguiente error: **IllegalArgumentException: 'requirement failed: Input column must be of type NumericType but got StringType'**

Para hacer un OneHotEncoder, equivalente a variable dummies, es necesarios pasar antes por _StringIndexer_. Ya hemos realizado esto, por favor recuerda la columna *make_indexed*.

Reutilizamos el ejemplo anterior:

In [ ]:
# Respuesta aqui

In [ ]:
cars_indexed.dtypes



Se aprecia que el dataframe `cars_indexed` ya incluye la variable `make_indexed` y es tipo numérica. Empezamos a trabajar a partir de aquí:

In [ ]:
# Respuesta aqui



### Pasar resultados a columnas independientes

Tanto al hacer el CountVectorizer como el OneHotEncoder, los resultados se encuentran en un vector en una sola columna. Sería muy útil separar los resultados en columnas distintas.

Veamos cómo hacerlo.



**Para el caso de CountVectorizer**

Un posible ejemplo podría ser generar una columna *doors_four* y una columna *doors_two*.

| num_of_doors   | doors_counter   |doors_four|doors_two|
| -------------: | -------------: | -------------:| -------------:|
| [four]| (2,[0],[1.0]) |1.0|0.0|
| [two,four]     | (2,[0,1],[1.0,1.0])| 1.0|1.0|

Para esto, primero se crea la columna '*activated_index*', transformando *doors_counter* a tipo Vector Array.

In [ ]:
# Respuesta aqui



Ahora debemos modificar el vector resultante, *activated_index*, para que cada elemento se encuentre en una columna distinta. También debemos saber los distintos valores/elementos sobre los que se ha hecho el count, esto se puede hacer mediante  *model_cv.vocabulary*

In [ ]:
# Respuesta aqui



Partimos nuestra columna 'activated_index' y renombramos las columnas resultantes con el tipo de evento correspondiente:

In [ ]:
# Respuesta aqui



¡Ya está hecho!




**Para el caso OneHotEncoder**

El proceso será equivalente con la diferencia de la procedencia de las distintas categorías.



Primero se crea una columna _ArrayType()_

In [ ]:
# Respuesta aqui



Modificar el vector resultante, *make_activated_index*, para que cada elemento se encuentre en una columna distinta



Debemos saber los distintos elementos sobre los que se ha hecho el count. La diferencia aquí es que se ha hecho un StringIndexer antes del OneHotEncoder y se debe volver a StringIndexer para recuperar las categorías.


In [ ]:
# Respuesta aqui



Al inspeccionar las categorias observamos que aparecen símbolos no permitidos. Esto debe a que existen macas de autos como "mercedes-benz". El guión medio "-" no esta permitido para los nombres de las columnas. Tomando esto en cuenta, partimos nuestra columna 'make_activated_index' en porciones y renombramos las columnas resultantes con la marca correspondiente:

In [ ]:
# Respuesta aqui



* Estudiamos comportamiento de OneHotEncoder

In [ ]:
# Respuesta aqui



La última categoría es 'mercury'. Veamos qué pasa:

In [ ]:
# Respuesta aqui



Se aprecia cómo 'make_mercury' toma valor nulo. De hecho, siempre la última columna toma el valor nulo.

In [ ]:
# Respuesta aqui



**¿Por qué?**

Porque OneHotEncoder supone que las columnas no nulas son las únicas categorías posibles para esa columna y por lo tanto, una de ellas es combinación lineal del resto. Por esta razón desestima la última de las categorías.

Hay situaciones de selección de variables donde todas deben estar presentes. Veamos como forzar la aparición de esta categoría también.

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui